<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Agrupamiento de Datos

En este laboratorio usaremos el DataFrame creado en la lección anterior resultado de la unión de los conjuntos de datos compartidos contigo. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Devuelve los resultados siempre en formato de DataFrame

In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Para guardar DataFrames en Excel
# -----------------------------------------------------------------------
from pandas import ExcelWriter

# Para generar todas las posibles combinaciones
# -----------------------------------------------------------------------
import itertools

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
# cargamos el dataframe del lab de EDA
df_merged = pd.read_csv("C:/Users/claud/GIT/laboratorio-groupby/datos/netflix_originals_merged.csv", index_col = 0)
df_merged.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,es_original
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,True
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN,NaN,NaN,NaN,NaN,False
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,NaN,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN,NaN,NaN,NaN,NaN,False
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,NaN,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN,NaN,NaN,NaN,NaN,False
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,NaN,NaN,NaN,NaN,NaN,False


#### Ejercicio 1: ¿Cuántos títulos hay por cada tipo de contenido ('Movie' o 'TV Show')?

In [3]:
titles_by_type = df_merged.groupby("type")["show_id"].count()

titles_by_type = titles_by_type.reset_index()
 
titles_by_type

,type,show_id
0,Movie,6131
1,TV Show,2676


#### Ejercicio 2: ¿Cuál es el promedio de puntuación de las películas agrupadas por clasificación de audiencia ('rating')? Interpreta los resultados, ¿hay algo que te llame la atención? Explica por qué. 

In [4]:
# Calcular el promedio de la puntuación (IMDB Score) de las películas agrupadas por clasificación de audiencia ('rating')
average_rating_by_audience = df_merged.groupby("rating")["IMDB Score"].mean().reset_index()

# Renombrar las columnas
average_rating_by_audience.columns = ["Rating", "Average IMDB Score"]

# Mostrar el DataFrame
average_rating_by_audience

,Rating,Average IMDB Score
0,66 min,NaN
1,74 min,NaN
2,84 min,NaN
3,G,6.500000
4,NC-17,NaN
5,NR,NaN
6,PG,6.800000
7,PG-13,6.386957
8,R,6.495745
9,TV-14,6.177660


1. Las clasificaciones con valores "NaN" no tienen una puntuación promedio debido a la falta de datos de puntuación para esas categorías (por ejemplo, "NC-17", "NR", "UR").
 
2. Las películas clasificadas como "G" (General Audience) tienen un promedio relativamente alto (6.50).
Clasificaciones más restrictivas como "PG-13" y "R" tienen puntajes promedio similares, con 6.39 y 6.50 respectivamente.
Curiosamente, en las películas dirigidas a audiencias jóvenes como "TV-Y" (6.78) tienden a tener puntajes más altos que aquellas para adultos como "TV-MA" (6.10).
 
3. Esto podría sugerir que hay películas para niños y familias que tienden a ser mejor recibidas, posiblemente por cumplir expectativas claras o por una mayor indulgencia de los críticos o la audiencia.

#### Ejercicio 3: ¿Cuántos títulos hay por cada país? Ordena los resultados de mayor a menor

In [5]:
# Agrupar por país y contar títulos, luego ordenar de mayor a menor
titles_by_country = df_merged.groupby("country")["show_id"].count().reset_index(name="title_count")

filtered_sorted_results = titles_by_country.sort_values(by="title_count", ascending=False)

# Mostrar el DataFrame
filtered_sorted_results

,country,title_count
603,United States,2818
251,India,972
506,United Kingdom,419
319,Japan,245
435,South Korea,199
...,...,...
22,"Australia, Iraq",1
743,Venezuela,1
744,"Venezuela, Colombia",1
746,West Germany,1


#### Ejercicio 4: ¿Cuál es la calificación promedio en IMDb de los títulos agrupados por tipo ('Movie' o 'TV Show')?

In [6]:
# Calcular la calificación promedio en IMDb agrupada por tipo ('Movie' o 'TV Show')
average_imdb_by_type = df_merged.groupby("type")["IMDB Score"].mean().reset_index()

# Renombrar columnas
average_imdb_by_type.columns = ["Type", "Average IMDB Score"]

# Mostrar el DataFrame resultante
average_imdb_by_type

,Type,Average IMDB Score
0,Movie,6.210916
1,TV Show,NaN


No podemos saber la calificación promedio en IMDb para TV Show porque hay valores nulos.

#### Ejercicio 5: ¿Cuántos títulos se añadieron a Netflix cada año? Para resolver este ejercicio tendrás uqe convertir la columna `date_added` a formato fecha usando el método `pd.to_datetime`. [Aquí](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) tienes la documentación, leeala con detenimiento antes de usar el método. 

In [7]:
df_merged.dtypes

show_id          object
type             object
title            object
director         object
cast             object
country          object
date_added       object
release_year      int64
rating           object
duration         object
listed_in        object
description      object
Genre            object
Premiere         object
Runtime         float64
IMDB Score      float64
Language         object
es_original        bool
dtype: object

In [8]:
# Cambiar columna "date_added" a tipo de dato "Datetime"
 
df_merged["date_added"] = pd.to_datetime(df_merged["date_added"], format="mixed")

In [9]:
df_merged.dtypes

show_id                 object
type                    object
title                   object
director                object
cast                    object
country                 object
date_added      datetime64[ns]
release_year             int64
rating                  object
duration                object
listed_in               object
description             object
Genre                   object
Premiere                object
Runtime                float64
IMDB Score             float64
Language                object
es_original               bool
dtype: object

In [10]:
# Contar el número de títulos añadidos cada año
titles_per_year = df_merged.groupby("date_added")["title"].count().reset_index()

# Mostrar el DataFrame resultante
titles_per_year

,date_added,title
0,2008-01-01,1
1,2008-02-04,1
2,2009-05-05,1
3,2009-11-18,1
4,2010-11-01,1
...,...,...
1709,2021-09-21,5
1710,2021-09-22,9
1711,2021-09-23,2
1712,2021-09-24,10


#### Ejercicio 6: ¿Cuál es el país con más títulos lanzados en el año 2020? Para resolver este ejercicio después del gropuby os puede resultar interesante el método [`idxmax`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html)

In [11]:
# Extraer el año de la columna 'date_added'
df_merged["year_added"] = df_merged["date_added"].dt.year

# Filtrar los datos para el año 2020
data_2020 = df_merged[df_merged["year_added"] == 2020]

# Agrupar por país y contar los títulos lanzados en 2020
titles_by_country_2020 = data_2020.groupby("country")["title"].count().reset_index(name="Title Count")

# Encontrar el país con más títulos lanzados en 2020
country_with_max_titles_2020 = titles_by_country_2020.loc[titles_by_country_2020["Title Count"].idxmax()]

# Convertir el resultado en un DataFrame
country_with_max_titles_2020_df = pd.DataFrame([country_with_max_titles_2020])

# Mostrar el resultado
country_with_max_titles_2020_df

,country,Title Count
192,United States,624


#### Ejercicio 7:¿Cuál es el promedio de puntuación en IMDb de los títulos agrupados por país?

In [12]:
# Calcular el promedio de puntuación en IMDb agrupado por país
average_imdb_by_country = df_merged.groupby("country")["IMDB Score"].mean().reset_index()

# Renombrar columnas para mayor claridad
average_imdb_by_country.columns = ["Country", "Average IMDB Score"]

# Mostrar los resultados
average_imdb_by_country

,Country,Average IMDB Score
0,", France, Algeria",NaN
1,", South Korea",NaN
2,Argentina,6.42
3,"Argentina, Brazil, France, Poland, Germany, De...",NaN
4,"Argentina, Chile",NaN
...,...,...
743,Venezuela,NaN
744,"Venezuela, Colombia",NaN
745,Vietnam,NaN
746,West Germany,NaN


#### Ejercicio 8: ¿Cuántos títulos fueron lanzados en cada año agrupados por tipo ('Movie' o 'TV Show')?

In [13]:
# Agrupar por año de lanzamiento y tipo ('Movie' o 'TV Show') y contar títulos
titles_per_year_type = df_merged.groupby(["release_year", "type"]).count().reset_index()

# Mostrar los resultados
titles_per_year_type

,release_year,type,show_id,title,director,cast,country,date_added,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,es_original,year_added
0,1925,TV Show,1,1,0,0,0,1,1,0,1,1,0,0,0,0,0,1,1
1,1942,Movie,2,2,2,1,2,2,2,2,2,2,0,0,0,0,0,2,2
2,1943,Movie,3,3,3,0,3,3,3,3,3,3,0,0,0,0,0,3,3
3,1944,Movie,3,3,3,1,3,3,3,3,3,3,0,0,0,0,0,3,3
4,1945,Movie,3,3,3,1,3,3,3,3,3,3,0,0,0,0,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2019,TV Show,397,397,31,350,348,397,397,86,397,397,0,0,0,0,0,397,397
115,2020,Movie,517,517,495,470,461,517,517,308,517,517,169,169,169,169,169,517,517
116,2020,TV Show,436,436,53,357,391,436,436,75,436,436,0,0,0,0,0,436,436
117,2021,Movie,277,277,252,244,169,277,277,157,277,277,56,56,56,56,56,277,277


#### Ejercicio 9: ¿Cuál es el país con la mayor cantidad de títulos de clasificación 'TV-MA'?

In [14]:
# Filtrar los datos para la clasificación 'TV-MA'
data_tv_ma = df_merged[df_merged["rating"] == "TV-MA"]

# Agrupar por país y contar los títulos con clasificación 'TV-MA'
titles_by_country_tv_ma = data_tv_ma.groupby("country").size().reset_index(name="Title Count")

# Seleccionar únicamente las columnas requeridas
titles_by_country_tv_ma_selected = titles_by_country_tv_ma[["country", "Title Count"]]

# Renombrar columnas para mayor claridad
titles_by_country_tv_ma_selected.columns = ["Country", "Title Count"]

# Mostrar los datos
titles_by_country_tv_ma_selected

,Country,Title Count
0,", South Korea",1
1,Argentina,39
2,"Argentina, Chile",1
3,"Argentina, Chile, Peru",1
4,"Argentina, France",1
...,...,...
309,"Uruguay, Argentina, Germany, Spain",1
310,"Uruguay, Argentina, Spain",1
311,"Uruguay, Spain, Mexico",1
312,Vietnam,3


#### Ejercicio 10. ¿Cuál es el promedio de duración de las películas agrupadas por país?

In [15]:
# Filtrar solo películas y agrupar por país
movie_data = df_merged[df_merged["type"] == "Movie"]
average_duration_by_country = movie_data.groupby("country")["Runtime"].mean().reset_index()

# Renombrar columnas para mayor claridad
average_duration_by_country.columns = ["Country", "Average Runtime (minutes)"]

# Mostrar los resultados al usuario
average_duration_by_country

,Country,Average Runtime (minutes)
0,", France, Algeria",NaN
1,Argentina,96.8
2,"Argentina, Brazil, France, Poland, Germany, De...",NaN
3,"Argentina, Chile",NaN
4,"Argentina, Chile, Peru",NaN
...,...,...
646,Venezuela,NaN
647,"Venezuela, Colombia",NaN
648,Vietnam,NaN
649,West Germany,NaN


Para Argentina el promedio es de 96,8 minutos. En los demás países salen valores nulos.

#### Ejercicio 11: ¿Cuál es el número total de títulos por país y clasificación de audiencia ('rating')?

In [16]:
# Contar el número total de títulos agrupados por país y clasificación de audiencia ('rating')
titles_by_country_and_rating = df_merged.groupby(["country", "rating"]).size().reset_index(name='Total Titles')

# Mostrar los primeros registros del resultado
titles_by_country_and_rating

,country,rating,Total Titles
0,", France, Algeria",TV-14,1
1,", South Korea",TV-MA,1
2,Argentina,NR,2
3,Argentina,R,1
4,Argentina,TV-14,7
...,...,...,...
1170,Vietnam,TV-14,3
1171,Vietnam,TV-G,1
1172,Vietnam,TV-MA,3
1173,West Germany,TV-MA,1


#### Ejercicio 12: ¿Cuántos títulos fueron lanzados antes del año 2000 agrupados por país?

In [17]:
# Filtrar títulos lanzados antes del año 2000
titles_before_2000 = df_merged[df_merged["release_year"] < 2000]

# Agrupar por país y contar los títulos
titles_before_2000_by_country = titles_before_2000.groupby("country").size().reset_index(name='Total Titles Before 2000')

# Mostrar los resultados
titles_before_2000_by_country.sort_values(by='Total Titles Before 2000', ascending=False).head()

,country,Total Titles Before 2000
47,United States,245
22,India,81
19,Hong Kong,29
43,United Kingdom,18
45,"United Kingdom, United States",12


#### Ejercicio 13: ¿Cuál es la puntuación promedio en IMDb de los títulos lanzados en 2021 agrupados por país?

In [18]:
# Filtrar los títulos lanzados en 2021
titles_2021 = df_merged[df_merged["release_year"] == 2021]

# Calcular la puntuación promedio en IMDb agrupada por país
imdb_avg_by_country_2021 = titles_2021.groupby("country")["IMDB Score"].mean().reset_index()

# Renombrar las columnas para mayor claridad
imdb_avg_by_country_2021.columns = ["Country", "Average IMDB Score (2021)"]

# Ordenar los resultados por puntuación promedio
imdb_avg_by_country_2021 = imdb_avg_by_country_2021.sort_values(by='Average IMDB Score (2021)', ascending=False)

# Mostrar los resultados
imdb_avg_by_country_2021.head()

,Country,Average IMDB Score (2021)
7,"Belgium, Netherlands",7.1
60,United Kingdom,7.1
30,"India, United States",7.1
22,"France, United States",6.5
54,Spain,6.4


#### Ejercicio 14: ¿Cuántos títulos fueron dirigidos por cada director? Ordena los resultados de mayor a menor?

In [19]:
# Contar el número de títulos únicos dirigidos por cada director
titles_by_director_unique = df_merged.groupby("director")["title"].count().reset_index()

# Renombrar las columnas para mayor claridad
titles_by_director_unique.columns = ["Director", "Total Titles Directed"]

# Ordenar de mayor a menor
titles_by_director_unique = titles_by_director_unique.sort_values(by='Total Titles Directed', ascending=False)

# Mostrar los resultados
titles_by_director_unique

,Director,Total Titles Directed
3392,Rajiv Chilaka,19
3443,"Raúl Campos, Jan Suter",18
4046,Suhas Kadav,16
2598,Marcus Raboy,16
1790,Jay Karas,14
...,...,...
4519,Álvaro Delgado-Aparicio L.,1
4518,Álvaro Brechner,1
4515,Zuko Nodada,1
4514,Zsolt Pálfi,1


#### Ejercicio 15: ¿Cuál es el lenguaje más frecuente en los títulos lanzados en los últimos 5 años agrupado por país?

In [20]:
from datetime import datetime

# Filtrar los títulos lanzados en los últimos 5 años
current_year = datetime.now().year
last_5_years = current_year - 5
recent_titles = df_merged[df_merged["release_year"] >= last_5_years]

# Agrupar por país y encontrar el lenguaje más frecuente en cada país
most_frequent_language = (
    recent_titles.groupby(["country", "Language"])["title"]
    .count()
    .reset_index(name='Title Count')
)

# Encontrar el lenguaje más frecuente para cada país
most_frequent_language_by_country = (
    most_frequent_language.loc[most_frequent_language.groupby("country")['Title Count'].idxmax()]
)

# Ordenar por país
most_frequent_language_by_country = most_frequent_language_by_country.sort_values(by="country")

# Mostrar los resultados
most_frequent_language_by_country.head()

,country,Language,Title Count
0,Argentina,Spanish,5
1,Austria,German,1
2,"Belgium, Netherlands",Dutch,1
3,Brazil,Portuguese,6
4,Canada,English,3


El lenguaje más frecuente de los títulos lanzados en los últimos 5 años es el Portuguese